# Introducción a la ingeniería de prompts

In [ ]:
# pip install groq
# pip install datasets

In [1]:
# import os
# from dotenv import load_dotenv
# load_dotenv("/home/santi/Repos/.env")

# groq_api_key = os.getenv("GROQ_API_KEY")

Ejecutar la siguiente celda en colab, tras haber creado un secreto con el nombre de `GROQ_API_KEY`

In [ ]:
# En Colab crear un Secreto con el nombre de GROQ_API_KEY 
from google.colab import userdata
groq_api_key = userdata.get('GROQ_API_KEY')

## ¿Qué es un Modelo de Lenguaje Grande?

Estamos rodeados de IA en estos días. Vamos a la fuente: ¿Qué dijo **ChatGPT** cuando se le preguntó "¿Qué es un modelo de lenguaje?"

| Tú | ChatGPT |
| :------------------------------ | :---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| ¿Qué es un modelo de lenguaje? | Un modelo de lenguaje es un tipo de sistema de inteligencia artificial que se entrena para entender y generar texto similar al humano. Aprende la estructura, gramática y semántica de un idioma procesando grandes cantidades de datos textuales. El objetivo principal de un modelo de lenguaje es predecir la probabilidad de la siguiente palabra o secuencia de palabras en un contexto dado. |

Eso es cierto, hasta donde llega. Pero la parte realmente interesante de la respuesta está oculta en la palabra **"grande."** El tamaño masivo de los modelos de lenguaje grandes es lo que ha impulsado la ola actual de progreso en este espacio. Para entender qué hace que estos modelos sean tan poderosos y cómo han evolucionado en los últimos años, hagamos un breve recorrido por la historia:

### Contexto Histórico de los Modelos de Lenguaje Grandes

* Alrededor de 2014, las **Recurrent Neural Networks (RNNs)** con una arquitectura de codificador-decodificador eran el estado del arte. Sin embargo, sufrían de un **cuello de botella informacional** entre estas dos etapas, limitando su capacidad para procesar secuencias largas de texto de manera efectiva. ¡Imagina tratar de recordar todo un párrafo después de leerlo solo una vez!

* En septiembre de 2014, el artículo *Neural Machine Translation by Jointly Learning to Align and Translate* introdujo el **Attention Mechanism**. Esto abordó el cuello de botella al permitir que el modelo "buscara suavemente" y se enfocara en la información más relevante de las etapas anteriores, en lugar de tratar de meter todo en una memoria limitada. Piensa en ello como resaltar los puntos clave en un documento largo.

* Esta innovación llevó a un cambio dramático en el campo. Para junio de 2017, los investigadores abandonaron las RNNs por completo porque, como decía el título de un artículo revolucionario, *Attention Is All You Need*. Este trabajo introdujo la **Transformer Architecture**, que resultó ser aún más poderosa para una amplia gama de tareas de NLP, y sigue siendo la base de muchos LLMs hoy en día.

* En junio de 2018, los investigadores refinaron aún más el Transformer con una técnica llamada *Improving Language Understanding by Generative Pre-Training*. Descubrieron que al "cortar el Transformer por la mitad" y usar solo el decodificador, podían crear un modelo que era notablemente efectivo en predecir la siguiente palabra en una secuencia. Este fue el nacimiento de **GPT-1 (Generative Pre-trained Transformer)**.

* Solo unos meses después, en febrero de 2019, **GPT-2** (*Language Models are Unsupervised Multitask Learners*) fue lanzado al mundo. Entrenado en un conjunto de datos diez veces más grande que su predecesor, GPT-2 mostró una capacidad asombrosa para realizar bien una variedad de tareas de NLP *sin* ningún ajuste específico. Este fue un momento decisivo: sugirió que simplemente entrenar un modelo de lenguaje en un conjunto de datos masivo podría desbloquear capacidades sorprendentes y potencialmente peligrosas.


| Fecha | Evento | Observación | Modelo |
| :--------- | :------------------------------------------------------------------------------------------------------------------ | :----------------------------------------------------------------------------------------------------------------------------------------------------------- | :----------- |
| c. 2014 | Las Recurrent Neural Networks dominaban las tareas de NLP. | El principal problema de estos modelos era un cuello de botella informacional que ocurría entre el codificador y el decodificador | |
| Sept 2014 | **Attention Mechanism** introducido en *Neural Machine Translation by Jointly Learning to Align and Translate*. | Este modelo usaba "soft search" para permitir un uso más eficiente de la información de contexto. | |
| Jun 2017 | Los investigadores abandonan las RNNs porque *Attention Is All You Need*. | Esto introdujo la **Transformer** architecture, que resultó ser más poderosa en muchas tareas de NLP y sigue siendo utilizada hoy en día. | |
| Jun 2018 | El Transformer se corta por la mitad usando la técnica *Improving Language Understanding by Generative Pre-Training*. | Solo se usa el decodificador del transformer en este caso. | **GPT-1** |
| Feb 2019 | **GPT-2** (*Language Models are Unsupervised Multitask Learners*) es entrenado en 10 veces la cantidad de datos de **GPT-1**. | En este punto, los investigadores encontraron que la **Generative Pre-Training** de modelos de lenguaje sin ajuste fino estaba produciendo resultados sorprendentes en múltiples tareas de NLP. | **GPT-2** |

Aquí tienes una muestra de lo que GPT-2 podía hacer:

| | |
| :------- | :------- |
| Missing word prediction | Summarization |
| Pronoun understanding | Sentiment analysis |
| Part of speech tagging | Entity extraction |
| Text compression | Question answering |
| | Translation |
| | Content generation |

Recuerda, logró todo esto *sin* estar específicamente entrenado para ninguna de estas tareas. Esto fue tanto emocionante como alarmante. Como los propios creadores escribieron en el artículo de lanzamiento de GPT-2:

> "*Nuestro modelo, llamado GPT-2 (un sucesor de GPT), fue entrenado simplemente para predecir la siguiente palabra en 40GB de texto de Internet. **Debido a nuestras preocupaciones sobre aplicaciones maliciosas de la tecnología, no estamos liberando el modelo entrenado.** (ref)*"

Esas preocupaciones eran muy reales. La capacidad de GPT-2 para generar texto que suena plausible planteó el espectro de un futuro inundado con:

* Artículos de noticias engañosos disfrazados de periodismo legítimo
* Suplantación de identidad y fraude en línea a gran escala
* Discurso de odio automatizado y propaganda en redes sociales
* Campañas sofisticadas de spam y phishing diseñadas para engañar incluso a los usuarios más experimentados

Esto nos lleva al meollo del asunto: ¿cómo aprovechamos el increíble poder de los LLMs mientras mitigamos los riesgos? La respuesta está en **prompt crafting**.



## Prompt Crafting: El Arte de Hablar con las Máquinas

Hemos establecido que los LLMs son increíblemente buenos en predecir la siguiente palabra. Pero, ¿cómo aprovechamos esa habilidad para realmente *hacer* cosas? Aquí tienes una forma simple de pensarlo:

> **Large Language Model Application = *Transformation Layer*** (entre el dominio del problema del usuario y el dominio del problema del LLM)

En este escenario, la aplicación actúa como un intermediario entre el usuario y el LLM, manejando toda la complejidad involucrada en la creación del prompt adecuado y procesando la respuesta del modelo para proporcionar información valiosa o completar las tareas solicitadas.

Vamos a explorar algunas técnicas específicas de prompt crafting:

In [46]:

from groq import Groq

client = Groq(
    api_key=groq_api_key,
)

def completion_groq(message, model="llama3-8b-8192"):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "you are  a base language model that only predict word by word. Avoid any comment additional to make a completion."
            },
            {
                "role": "user",
                "content": message,
            }
        ],
        model=model,
        temperature=1,
    )
    return chat_completion.choices[0].message.content

message = "Explain the importance of fast language models"
response = completion_groq(message)
print(response)

state



### Técnica 1: Few-Shot Prompting

El avance de los LLMs no fue solo su tamaño; fue el descubrimiento de que realmente podíamos controlar su salida estructurando cuidadosamente la entrada. **Few-shot prompting** es un ejemplo perfecto de esto. Es como darle al LLM un curso intensivo en un nuevo tema proporcionando unos pocos ejemplos bien elegidos. Así es como funciona para una aplicación de traducción hipotética:

```markdown
> How are you doing today?
< ¿Cómo estás hoy?
> My name is John.
< Mi nombre es John.
> Can I have fries with that?
```

Le damos al modelo dos traducciones completas de inglés a español, estableciendo un patrón claro. Luego le presentamos una nueva oración en inglés. El LLM, entrenado para predecir la siguiente palabra, "entiende" que debe continuar el patrón y proporcionar la traducción al español de "Can I have fries with that?"

Puntos clave sobre few-shot prompting:

* **Simplicidad y Poder:** Es notablemente efectivo para dirigir las respuestas de los LLM hacia resultados deseados.
* **Más allá de los Ejemplos:** El encabezado, la estructura e incluso el uso de construcciones de lenguaje familiares (como markdown) influyen en el comportamiento del LLM.
* **El Contexto es Clave:** Cuanta más información relevante proporcionemos al principio, mayores serán las posibilidades del LLM de generar una respuesta útil.


In [38]:
message = """Q: How are you doing today?
A: ¿Cómo estás hoy?
Q: My name is John.
A: Mi nombre es John.
Q: Can I have fries with that?
A:"""

print(completion_groq(message))


Puedo tener papas fritas con eso?



### Técnica 2: Chain of Thought Reasoning

Aunque los LLMs sobresalen en tareas que implican fluidez en el lenguaje y reconocimiento de patrones, tradicionalmente han tenido dificultades con problemas que requieren razonamiento lógico o sentido común. Pregunta a un LLM temprano un problema matemático simple, y podrías obtener una respuesta sin sentido:

```markdown
P: A un panadero le lleva una hora hacer un pastel. ¿Cuánto tiempo tardan 3 panaderos en hacer 3 pasteles?
R: 3
```

In [48]:
message = """Q: A un panadero le lleva una hora hacer un pastel. ¿Cuánto tiempo tardan 3 panaderos en hacer 3 pasteles?
A:"""

print(completion_groq(message,model="gemma-7b-it"))

0.33 horas.



**Chain-of-thought reasoning** aborda esta limitación al pedir explícitamente al LLM que descomponga el problema en una serie de pasos lógicos. Es como mostrar tu trabajo en un examen de matemáticas, guiando al LLM hacia una solución más precisa. Así es como se ve en la práctica:

```markdown
P: Jim tiene el doble de la edad de Steve. Jim tiene 12 años, ¿cuántos años tiene Steve?
R: En forma de ecuación: 12 = 2*a donde a es la edad de Steve. Dividiendo ambos lados por 2 vemos que a = 6. Steve tiene 6 años.
P: A un panadero le lleva una hora hacer un pastel. ¿Cuánto tiempo tardan 3 panaderos en hacer 3 pasteles?
R:
```

Al proporcionar un ejemplo trabajado, hemos dado al LLM un marco para abordar el segundo problema lógicamente. Una respuesta mucho más probable (y correcta) sería:

```markdown
P: Jim tiene el doble de la edad de Steve. Jim tiene 12 años, ¿cuántos años tiene Steve?
R: En forma de ecuación: 12 = 2*a donde a es la edad de Steve. Dividiendo ambos lados por 2 vemos que a = 6. Steve tiene 6 años.
P: A un panadero le lleva una hora hacer un pastel. ¿Cuánto tiempo tardan 3 panaderos en hacer 3 pasteles?
R: Si cada panadero tarda 1 hora en hacer un pastel, entonces 3 panaderos tardarán 1 hora en hacer 3 pasteles
```

In [52]:
message = """Q: Jim tiene el doble de la edad de Steve. Jim tiene 12 años, ¿cuántos años tiene Steve?
A: En forma de ecuación: 12 = 2*a donde a es la edad de Steve. Dividiendo ambos lados por 2 vemos que a = 6. Steve tiene 6 años.
Q: A un panadero le lleva una hora hacer un pastel. ¿Cuánto tiempo tardan 3 panaderos en hacer 3 pasteles?
A:"""

print(completion_groq(message, model="gemma-7b-it"))

1 hora.

Cada panadero puede hacer un pastel en 1 hora, así que 3 panaderos pueden hacer 3 pasteles en 1 hora.


**Perspectivas Clave sobre Chain-of-Thought Prompting:**

* <font color='blue'>**Reemplazando el Monólogo Interno:**</font> Los humanos razonan a través de problemas con diálogo interno. Los LLMs, por otro lado, dependen de la predicción de la siguiente palabra. Chain-of-thought prompting les obliga a externalizar ese proceso de razonamiento, llevando a resultados más lógicos.
* <font color='green'>**Aprovechando el Attention Mechanism:**</font> El attention mechanism en los LLMs actúa como un "espacio de trabajo" donde estos pasos intermedios de razonamiento pueden ser expuestos y referenciados.
* <font color='red'>**Previniendo el "Sangrado":**</font> Los ejemplos de few-shot pueden a veces "sangrar" en la salida, introduciendo sesgos no deseados. Podemos mitigar esto instruyendo explícitamente al LLM a "pensar paso a paso" en lugar de esperar que complete el patrón literalmente.
* <font color='purple'>**La Concisión es Clave:**</font> Los LLMs tienen una ventana de contexto limitada. Las cadenas de pensamiento excesivamente largas pueden ser contraproducentes, ya que el modelo podría perder de vista el problema original.


#### Zero shot COT

```
Q: A un panadero le lleva una hora hacer un pastel. ¿Cuánto tiempo tardan 3 panaderos en hacer 3 pasteles?

A: **Pensemos paso a paso**. *El tiempo que se tarda en hornear un pastel es el mismo sin importar cuántos pasteles se hagan y cuántas personas trabajen en ellos.
Por lo tanto, la respuesta sigue siendo 1*
```

In [60]:
# EJERCICIO 1: implementa el ejemplo de zero COT anterior usando la API de groq con el modelo gemma-7b-it y la función completion_groq.
message = """Q: A un panadero le lleva una hora hacer un pastel. ¿Cuánto tiempo tardan 3 panaderos en hacer 3 pasteles?
A: **Pensemos paso a paso**. """

print(completion_groq(message, model="gemma-7b-it"))

1 hora. 

Si un panadero hace un pastel en una hora, 3 panaderos trabajan juntos pueden hacer 3 pasteles en la misma cantidad de tiempo.



### Técnica 3: Imitación de Documentos

Imagina encontrar un trozo de papel rasgado con estas palabras:

> <font color='yellow'>*My cable is out.  And I am going to miss the Super Bowl*</font>

Incluso con un contexto limitado, puedes inferir mucho. Algo sucedió (corte de cable), llevando a una consecuencia negativa (perder el Super Bowl). Esta es la esencia de **document mimicry**. Los LLMs están entrenados en conjuntos de datos masivos de texto, código y otra información estructurada. Al imitar los formatos y convenciones de documentos comunes, podemos ayudar a los LLMs a "entender" el contexto de nuestros prompts y generar respuestas más relevantes.

El LLM podría tener dificultades para proporcionar una respuesta útil. Pero, ¿qué pasa si estructuramos el prompt así:

```markdown
# IT Support Assistant
The following is a transcript between an award-winning
IT support rep and a customer.

## Customer: 
My cable is out! And I'm going to miss the Super Bowl!

## Support Assistant:
```

Al usar encabezados de markdown, un formato de transcripción e incluso un lenguaje cargado emocionalmente, hemos dado al LLM una comprensión mucho más rica de la situación. Ahora sabe:

* <font color='blue'>**Tipo de Documento:**</font> Esta es una interacción de soporte al cliente
* <font color='green'>**Estructura:**</font> Hay un diálogo claro entre un cliente y un agente de soporte
* <font color='red'>**Contexto Emocional:**</font> El cliente está frustrado y necesita ayuda

Esta información adicional aumenta significativamente las posibilidades de que el LLM genere una respuesta útil y empática.



Aquí tienes una comparación rápida de las técnicas que hemos cubierto:

| Técnica | Descripción |
| :--------------------------------------------------------- | :------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| Few-Shot Prompting | Ejemplos hechos a mano para "entrenar" al LLM a completar tareas similares |
| Chain of Thought Prompting | Usa ejemplos de few-shot, pero el objetivo principal es guiar al modelo en los pasos a seguir para llegar a la respuesta correcta a problemas que implican lógica y razonamiento |
| Document Mimicry | Usa elementos estructurales y formato textual de documentos comunes como transcripciones y markdown. El LLM, al estar entrenado en millones de documentos y códigos, debería ser capaz de entenderlos, mejorando sus capacidades para generar respuestas relevantes. |


#### Intuición para la Creación de Prompts: Los LLMs son Humanos Mecánicos Tontos.

* Los LLMs entienden mejor cuando usas un lenguaje y construcciones familiares.
* Los LLMs se distraen. No llenes el prompt con mucha información "por si acaso".
* Los LLMs no son psíquicos. Si la información no está en el entrenamiento o en el prompt, entonces no la saben.
* Si miras el prompt y no puedes entenderlo, un LLM es inútil.

### Construyendo aplicaciones con LLMs

<img style="width: 70%" src="images/fig-14.png" id="llm-app"/>

### Creando el prompt

* Recopilar contexto
* Rankear contexto
* Recortar contexto
* Ensamblar el prompt


**Creando el Prompt para la Compleción de Código**

Apliquemos este concepto a un ejemplo más técnico: la compleción de código. Imagina un IDE (entorno de desarrollo integrado) potenciado por un LLM que ayuda a los desarrolladores a escribir código más rápido. Cuando un usuario solicita la compleción de código, el IDE no solo envía al LLM la línea de código actual. 

<img style="width: 70%" src="images/fig-16.png" id="llm-app"/>

Este prompt, estructurado como un archivo de código con comentarios, ayuda al LLM a "entender" la base de código y generar compleciones más relevantes.



### La introducción del Chat

El lanzamiento de ChatGPT en noviembre de 2022 marcó un punto de inflexión significativo en la evolución de las aplicaciones de LLM. ChatGPT introdujo una nueva forma de interactuar con los LLMs, utilizando un formato conversacional y un lenguaje estructurado llamado ChatML.

<img style="width: 70%" src="images/fig-17.png" id="llm-app"/>

Uno de los beneficios clave de ChatML es la capacidad de usar **system messages** para guiar el comportamiento del LLM y prevenir salidas indeseables. Las system messages son instrucciones del desarrollador que dictan el tono, estilo y parámetros de seguridad del asistente.

### La introducción de las Tools

<img style="width: 70%" src="images/fig-18.png" id="llm-app"/>

El prompt anterior de ChatML define claramente los roles del usuario y del asistente, y ahora se introduce un nuevo elemento: `tool` o también llamado *function calling*. Aquí es donde entra en juego **function calling**. Los LLMs equipados con esta capacidad ahora pueden interactuar con herramientas y APIs externas, ampliando significativamente sus aplicaciones potenciales. En este ejemplo, el LLM reconoce la solicitud del usuario de información meteorológica y usa la función `get_weather` para recuperar los datos. La herramienta devuelve la temperatura (digamos 78 grados Fahrenheit), y el LLM incorpora esta información en su respuesta.

### Construyendo Aplicaciones con LLMs: Bags of Tools Agent

<img style="width: 70%" src="images/fig-19.png" id="llm-app"/>


<img style="width: 70%" src="images/fig-20.png" id="llm-app"/>

Esto destaca un punto crucial: **incluso con function calling, la aplicación sigue siendo la capa de transformación.** Es responsable de mediar entre la intención del usuario, las capacidades del LLM y el mundo exterior, asegurando interacciones seguras y apropiadas.


### Tips para definir las Tools

* No tengas "demasiadas" tools - busca evidencia de colisiones

* Nombra las tools de forma simple y clara (¿y en formato TypeScript?)

* No copies/pegues tu API - mantén los argumentos simples y pocos

* Mantén cortas las descripciones de la función y los argumentos y considera lo que el modelo
sabe
    - Probablemente entiende la documentación pública.
    - No conoce los acrónimos internos de la empresa.

* Más sobre los argumentos
    - Los argumentos anidados no retienen las descripciones
    - Puedes usar enum y default, pero no minimum, maximum...
    
* Salida de la habilidad – no incluyas contenido extra "por si acaso"
* Errores de la habilidad – cuando sea razonable, envía errores al modelo (errores de validación)


**Conclusiones Clave de ChatGPT y Function Calling:**

* **ChatML:** Un lenguaje estructurado para construir aplicaciones conversacionales con LLM, facilitando la definición de roles, el control del flujo y la gestión del contexto.
* **Function Calling:** Permite que los LLMs interactúen con herramientas y APIs externas, ampliando significativamente sus aplicaciones prácticas.
* **System Messages:** Permiten a los desarrolladores imponer directrices de seguridad, establecer el tono del asistente y prevenir salidas no deseadas.
* **Capa de Transformación:** A pesar de estos avances, la aplicación sigue jugando un papel vital en la traducción entre las necesidades del usuario y las capacidades del LLM, asegurando un uso responsable y ético.



### Una Perspectiva Más Amplia sobre Técnicas de Prompt Engineering

Aunque las técnicas que hemos discutido proporcionan una base sólida, es esencial reconocer el panorama más amplio del prompt engineering:

* **Zero-Shot Prompting:** Proporcionar al LLM una descripción de la tarea y esperar que la realice sin ningún ejemplo. Sorprendentemente efectivo para algunas tareas como la summarization.
* **One-Shot Prompting:** Similar al zero-shot, pero con un solo ejemplo proporcionado.
* **Instruction Prompting:** Guiar al LLM para que responda en un tono, persona, longitud o formato específico.


Recuerda, estas técnicas están en constante evolución junto con la tecnología de los LLM. Lo que hoy parece innovador, mañana podría ser común.

[https://www.promptingguide.ai/](https://www.promptingguide.ai/) es una guía de ing. de prompts con múltiples técnicas. Es recomendable su lectura.


## Conclusión

El prompt crafting es tanto un arte como una ciencia. Al entender las fortalezas y limitaciones de los LLMs, y al aprovechar técnicas como few-shot prompting, chain-of-thought reasoning, document mimicry, y el poder de ChatML y function calling, podemos desbloquear el potencial transformador de esta tecnología mientras navegamos por las consideraciones éticas de manera responsable.

El futuro de la IA depende de nuestra capacidad para comunicarnos eficazmente con estas poderosas máquinas, guiándolas hacia un futuro que beneficie a toda la humanidad.

## Estructuración de salidad en modo JSON

In [69]:
def chat_groq(user, system="you are a helpful assistant",  model="llama3-8b-8192", temperature=1, **kwargs):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": system
            },
            {
                "role": "user",
                "content": user,
            }
        ],
        model=model,
        temperature=temperature,
        response_format={"type": "json_object"}
    )
    return chat_completion.choices[0].message.content

print(chat_groq(system="eres un asistente que siempre habla en español y que responde en formato JSON",
                 user="hola como estas?"))

{
"message": "Hola! Estoy funcionando correctamente, gracias por preguntar.",
"status": " Bien"
}


In [85]:
texto = "El prompt crafting es tanto un arte como una ciencia. Al \
    entender las fortalezas y limitaciones de los LLMs, y al aprovechar técnicas como few-shot prompting,\
          chain-of-thought reasoning, document mimicry, y el poder de ChatML y function calling,\
              podemos desbloquear el potencial transformador de esta tecnología mientras navegamos\
                  por las consideraciones éticas de manera responsable.\
                    El futuro de la IA depende de nuestra capacidad para comunicarnos eficazmente\
                          con estas poderosas máquinas, guiándolas hacia un futuro que beneficie a toda la humanidad."

respuesta = chat_groq(
            system="""extrae las palabras en inglés del texto. Genera la respuesta en un formato JSON
            de la forma:
            ```json
            {"english_words": ["word1", "word2", ...]}
            ```""",
            user=f"Este es el texto: {texto}",
            model="llama3-70b-8192",
            temperature=0
        )
print(respuesta)

{"english_words": ["prompt", "crafting", "LLMs", "few-shot", "prompting", "chain-of-thought", "reasoning", "document", "mimicry", "ChatML", "function", "calling", "transformador", "IA"]}


## Tool Calling en Groq (con nuevos modelos, es muy reciente!)

Es un ejemplo que está en la [documentación de groq](https://console.groq.com/docs/tool-use)

In [88]:
from groq import Groq
import json

client = Groq()
MODEL = 'llama3-groq-70b-8192-tool-use-preview'

def calculate(expression):
    """Evaluate a mathematical expression"""
    try:
        result = eval(expression)
        return json.dumps({"result": result})
    except:
        return json.dumps({"error": "Invalid expression"})

def run_conversation(user_prompt, verbose=False):
    if verbose:
        print("Starting conversation with user prompt:", user_prompt)
        
    messages=[
        {
            "role": "system",
            "content": "You are a calculator assistant. Use the calculate function to perform mathematical operations and provide the results."
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]
    
    if verbose:
        print("\nInitial messages:", messages)
        
    tools = [
        {
            "type": "function",
            "function": {
                "name": "calculate",
                "description": "Evaluate a mathematical expression",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "expression": {
                            "type": "string",
                            "description": "The mathematical expression to evaluate",
                        }
                    },
                    "required": ["expression"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto",
        max_tokens=4096
    )

    response_message = response.choices[0].message
    if verbose:
        print("\nFirst Response message:", response_message)
    tool_calls = response_message.tool_calls
    if tool_calls:
        available_functions = {
            "calculate": calculate,
        }
        messages.append(response_message)
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                expression=function_args.get("expression")
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
        if verbose:
            print("\nMessages after tool calls:", messages)
        second_response = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )
        return second_response.choices[0].message.content

user_prompt = "What is 25 * 4 + 10?"
print("\n",run_conversation(user_prompt, verbose=True))

Starting conversation with user prompt: What is 25 * 4 + 10?

Initial messages: [{'role': 'system', 'content': 'You are a calculator assistant. Use the calculate function to perform mathematical operations and provide the results.'}, {'role': 'user', 'content': 'What is 25 * 4 + 10?'}]

First Response message: ChoiceMessage(content=None, role='assistant', tool_calls=[ChoiceMessageToolCall(id='call_8veg', function=ChoiceMessageToolCallFunction(arguments='{"expression": "25*4+10"}', name='calculate'), type='function')])

Messages after tool calls: [{'role': 'system', 'content': 'You are a calculator assistant. Use the calculate function to perform mathematical operations and provide the results.'}, {'role': 'user', 'content': 'What is 25 * 4 + 10?'}, ChoiceMessage(content=None, role='assistant', tool_calls=[ChoiceMessageToolCall(id='call_8veg', function=ChoiceMessageToolCallFunction(arguments='{"expression": "25*4+10"}', name='calculate'), type='function')]), {'tool_call_id': 'call_8veg'

La lógica del código anterior en el contexto de function calling o tools:

1. Se hace una llamada inicial al modelo, proporcionando los mensajes, las herramientas disponibles (`tools`), y configurando `tool_choice="auto"`. Esto permite al modelo decidir si necesita usar una herramienta.

2. Se extrae el mensaje de respuesta y, si hay llamadas a herramientas (`tool_calls`), se procede a ejecutarlas.

3. Se define un diccionario `available_functions` que mapea el nombre de la función ("calculate") a la función Python real.

4. Para cada llamada a herramienta:
   - Se obtiene el nombre de la función y los argumentos.
   - Se ejecuta la función correspondiente con los argumentos proporcionados.
   - Se añade la respuesta de la función a la lista de mensajes.

5. Después de ejecutar todas las llamadas a herramientas, se hace una segunda llamada al modelo, incluyendo los resultados de las herramientas en los mensajes.

6. Finalmente, se devuelve el contenido del mensaje de esta segunda respuesta.

Esta implementación permite al modelo:
1. Reconocer cuando necesita realizar un cálculo.
2. Solicitar la ejecución de la función `calculate`.
3. Recibir el resultado del cálculo.
4. Usar ese resultado para formular una respuesta final al usuario.

Es un ejemplo claro de cómo el function calling o tools permiten a los modelos de lenguaje interactuar con funciones externas para realizar tareas específicas, en este caso, cálculos matemáticos.